# Context


This notebook summarizes the steps in the development process for our survey model.

### The data

The model leverages a [public dataset](https://wwwn.cdc.gov/nchs/nhanes/default.aspx) from the National Health and Nutrition Examination Survey (NHANES). NHANES is a national survey that monitors the health and nutrional status of adults and children across the US. NHANCES is run by the National Center for Health Statistics (NCHS). NCHS is part of centers for disease control and prevention (CDC) and is responsible for producing vital and health statistics for the nation. The public database contains data to 40+ surveys that were each conducted 12 times between 1999-2020. 

### EDA & Data Pipeline

We explored roughly half of the survey files available in the database. For our final model we are using {XXX} different survey files spanning years of {YYY}. Our repository contains links to our jupyter notebooks used for creating our data-pipeline {ZZZ} & performing exploratory data analysis (EDA) {AAA}. 

### Model Development

#### Model Objective

The objective of our model is to predict whether a survey respondent is depressed. 
- The outcome variable is binary indicator variable called "MDD". The value is 1 if the individual has been diagnosed & is taking medication related to depression. 
- The input variables include data around demographic traits, medical & physical health, mood/behavioral data.

#### Training Phase

We explored several options during the training phase in an effort to build the best model and better understand our target sample.


1) We explored the following 6 classifiers for our model;
- Logistic Regression: XXX
- Random Forest: XXX
- Decision Tree: XXX
- K-Nearest Neighbor (KNN): XXX
- Naive Bayes: XXX
- Gradient Boosting Classifier: XXX

2) Positive class imbalance: Across our entire dataset ~4% of respondents have MDD=1. Among preganant woman, ~10% have MDD=1. We explored 3 stragies to account for this;
- SMOTE: XXX
- Test/Train Proportion: XXX
- Adding additional survey years: XXX
 
3) Segmentation: Our project is oriented around predicting PPD, hence our model focuses on predicting depression among woman that have been pregnant. We did however want to compare performance when across several groups; A) all respondents B) males only C) females only D) females that have been preganant only. This is used to understand potential differences in predicting depression among different audiences.

#### Test Phase



- Evaluation criteria: 
- Feature selection:
- Hyperparameters: 
- Error analysis:

# Model Development

## Setup

### Load Packages

### Load Data

### Candidate Features

### Function to create test & train dataset

### Function to get performance across different models

### Function for Error analysis

## Baseline Model

## Error Analysis